In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/sarscov2-ctscan-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (1153).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (352).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (179).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (672).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (501).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (405).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (436).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (883).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (564).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (936).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (837).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (421).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (1031).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVID/Non-Covid (221).png
/kaggle/input/sarscov2-ctscan-dataset/non-COVI

In [3]:
!pip install split-folders
import tensorflow as tf
import cv2, pathlib, splitfolders
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [16]:
epochs = 10
batch_size = 32
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)

In [6]:
def create_data_binary(data_bs):
    data_bs = pathlib.Path(data_bs)
    splitfolders.ratio(data_bs, output='Imgs/', seed=1234, ratio=(0.7, 0.15, 0.15), group_prefix=None)
    data_gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range = 10,
    width_shift_range = 0.1, 
    height_shift_range = 0.1,
    samplewise_center = True,
    samplewise_std_normalization = True 
    )
    train_ds = data_gen.flow_from_directory('Imgs/train/', target_size=(img_height, img_width),
                                            class_mode='binary', batch_size=batch_size, subset='training')
    test_ds = data_gen.flow_from_directory('Imgs/test/', target_size=(img_height, img_width),
                                          class_mode='binary', batch_size=batch_size, shuffle=False)

    return train_ds ,test_ds
train_data, test_data = create_data_binary('/kaggle/input/sarscov2-ctscan-dataset/')

Copying files: 2481 files [00:14, 171.17 files/s]

Found 1736 images belonging to 2 classes.


Found 374 images belonging to 2 classes.


In [8]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten,Conv2D,Dropout,BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import Adam
import tensorflow as tf

# Load ResNet50
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

94765736/94765736 [==============================] - 0s 0us/step


In [9]:
# Add new classifier layers on top of the pre-trained model
x = tf.keras.layers.GlobalAveragePooling2D()(resnet.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.7)(x)
x = tf.keras.layers.Dense(512, activation = 'LeakyReLU')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(64, activation = 'LeakyReLU')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
model_resnet = tf.keras.models.Model(
    inputs = resnet.input, 
    outputs = x
)
model_resnet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [10]:
for layer in model_resnet.layers[:120]:
    layer.trainable = False

In [11]:
optimizer = Adam(learning_rate= 0.001)
model_resnet.compile(optimizer=optimizer, loss = 'binary_crossentropy' ,metrics=['accuracy'])

In [17]:
# Train the models
history_resnet = model_resnet.fit(train_data,
                       validation_data = test_data,
                       epochs=epochs,batch_size= 8,
                       callbacks = [
                           tf.keras.callbacks.EarlyStopping(
                               patience = 5, monitor = 'val_loss', mode = 'min', 
                               restore_best_weights = True
                           ),
                           tf.keras.callbacks.ReduceLROnPlateau(
                               patience = 2, monitor = 'val_loss', 
                               mode = 'min', factor = 0.1
                           )
                       ])

Epoch 1/10
55/55 [==============================] - 36s 650ms/step - loss: 0.3927 - accuracy: 0.8209 - val_loss: 1.0812 - val_accuracy: 0.7647 - lr: 0.0010
Epoch 2/10
55/55 [==============================] - 36s 646ms/step - loss: 0.3755 - accuracy: 0.8312 - val_loss: 0.4241 - val_accuracy: 0.8342 - lr: 0.0010
Epoch 3/10
55/55 [==============================] - 36s 651ms/step - loss: 0.3334 - accuracy: 0.8537 - val_loss: 0.4045 - val_accuracy: 0.8369 - lr: 0.0010
Epoch 4/10
55/55 [==============================] - 36s 654ms/step - loss: 0.2779 - accuracy: 0.8819 - val_loss: 0.3460 - val_accuracy: 0.8904 - lr: 0.0010
Epoch 5/10
55/55 [==============================] - 36s 656ms/step - loss: 0.2845 - accuracy: 0.8819 - val_loss: 0.3613 - val_accuracy: 0.8235 - lr: 0.0010
Epoch 6/10
55/55 [==============================] - 35s 640ms/step - loss: 0.2579 - accuracy: 0.8929 - val_loss: 0.4142 - val_accuracy: 0.8717 - lr: 0.0010
Epoch 7/10
55/55 [==============================] - 35s 644ms/st

In [18]:
loss_resnet, acc_resnet = model_resnet.evaluate(test_data)

12/12 [==============================] - 7s 536ms/step - loss: 0.1817 - accuracy: 0.9251
